In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
conf = SparkConf().setAppName('umsi618f22lab7').set("spark.hadoop.validateOutputSpecs", "false")
sc = SparkContext.getOrCreate(conf = conf)
sqlc = SQLContext(sc)

/sw/pkgs/arc/spark/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [178]:
input_file = sc.textFile("movies_metadata_2.csv")
input_file = input_file.flatMap(lambda line: [line.split(",")[:8]])
input_file = input_file.filter(lambda line: line[0].isnumeric() and line[3].isnumeric() and line[4].isnumeric() and line[5].isnumeric() and line[6].isnumeric() and line[7].isnumeric())
metadata_df = input_file.toDF()
metadata_df.registerTempTable("metadata_df")
metadata_df.show(20)
metadata_df.count()

+--------+--------------------+----------+---------+---+---+---+----+
|      _1|                  _2|        _3|       _4| _5| _6| _7|  _8|
+--------+--------------------+----------+---------+---+---+---+----+
|30000000|           Toy Story|10/30/1995|373554033| 81| 10| 30|1995|
|65000000|             Jumanji|12/15/1995|262797249|104| 12| 15|1995|
|       0|    Grumpier Old Men|12/22/1995|        0|101| 12| 22|1995|
|16000000|   Waiting to Exhale|12/22/1995| 81452156|127| 12| 22|1995|
|       0|Father of the Bri...| 2/10/1995| 76578911|106|  2| 10|1995|
|60000000|                Heat|12/15/1995|187436818|170| 12| 15|1995|
|58000000|             Sabrina|12/15/1995|        0|127| 12| 15|1995|
|       0|        Tom and Huck|12/22/1995|        0| 97| 12| 22|1995|
|35000000|        Sudden Death|12/22/1995| 64350171|106| 12| 22|1995|
|58000000|           GoldenEye|11/16/1995|352194034|130| 11| 16|1995|
|62000000|The American Pres...|11/17/1995|107879496|106| 11| 17|1995|
|       0|Dracula: D

43098

In [180]:
metadata_df = sqlc.sql("""
SELECT  
_2 as Title, 
_3 as Release_Date, 
CAST(_1 as int) as Budget,
CAST(_4 as int) as Revenue,
CAST(_4 as int)-CAST(_1 as int) as Profit,
CAST(_5 as int) as Run_Time,
CAST(_6 as int) as Month,
CAST(_7 as int) as Day,
CAST(_8 as int) as Year
FROM metadata_df
""")
metadata_df.show()
metadata_df.count()

+--------------------+------------+--------+---------+---------+--------+-----+---+----+
|               Title|Release_Date|  Budget|  Revenue|   Profit|Run_Time|Month|Day|Year|
+--------------------+------------+--------+---------+---------+--------+-----+---+----+
|           Toy Story|  10/30/1995|30000000|373554033|343554033|      81|   10| 30|1995|
|             Jumanji|  12/15/1995|65000000|262797249|197797249|     104|   12| 15|1995|
|    Grumpier Old Men|  12/22/1995|       0|        0|        0|     101|   12| 22|1995|
|   Waiting to Exhale|  12/22/1995|16000000| 81452156| 65452156|     127|   12| 22|1995|
|Father of the Bri...|   2/10/1995|       0| 76578911| 76578911|     106|    2| 10|1995|
|                Heat|  12/15/1995|60000000|187436818|127436818|     170|   12| 15|1995|
|             Sabrina|  12/15/1995|58000000|        0|-58000000|     127|   12| 15|1995|
|        Tom and Huck|  12/22/1995|       0|        0|        0|      97|   12| 22|1995|
|        Sudden Death

43098

In [181]:
metadata_df_1 = sqlc.sql("""
SELECT  
metadata_df._2 as Title, 
metadata_df._3 as Release_Date, 
CAST(metadata_df._1 as int) as Budget,
CAST(metadata_df._4 as int) as Revenue,
CAST(metadata_df._4 as int)-CAST(metadata_df._1 as int) as Profit,
CAST(metadata_df._5 as int) as Run_Time,
(CASE WHEN CAST(metadata_df._7 as int) > 15 THEN CAST(metadata_df._6 as int)+1 ELSE CAST(metadata_df._6 as int) END) as Adjusted_Month,
CAST(metadata_df._7 as int) as Day,
CAST(metadata_df._8 as int) as Year
FROM metadata_df
""")
metadata_df_1.show()
metadata_df.count()

+--------------------+------------+--------+---------+---------+--------+--------------+---+----+
|               Title|Release_Date|  Budget|  Revenue|   Profit|Run_Time|Adjusted_Month|Day|Year|
+--------------------+------------+--------+---------+---------+--------+--------------+---+----+
|           Toy Story|  10/30/1995|30000000|373554033|343554033|      81|            11| 30|1995|
|             Jumanji|  12/15/1995|65000000|262797249|197797249|     104|            12| 15|1995|
|    Grumpier Old Men|  12/22/1995|       0|        0|        0|     101|            13| 22|1995|
|   Waiting to Exhale|  12/22/1995|16000000| 81452156| 65452156|     127|            13| 22|1995|
|Father of the Bri...|   2/10/1995|       0| 76578911| 76578911|     106|             2| 10|1995|
|                Heat|  12/15/1995|60000000|187436818|127436818|     170|            12| 15|1995|
|             Sabrina|  12/15/1995|58000000|        0|-58000000|     127|            12| 15|1995|
|        Tom and Huc

43098

In [182]:
metadata_df_2 = sqlc.sql("""
SELECT Title,Release_Date,Budget,Revenue,Profit,Run_Time,Adjusted_Month_2 as Month,Adjusted_Year as Year FROM
(SELECT  
metadata_df._2 as Title, 
metadata_df._3 as Release_Date, 
CAST(metadata_df._1 as int) as Budget,
CAST(metadata_df._4 as int) as Revenue,
CAST(metadata_df._4 as int)-CAST(metadata_df._1 as int) as Profit,
CAST(metadata_df._5 as int) as Run_Time,
CAST(_6 as int) as Month,
(CASE WHEN CAST(metadata_df._7 as int) > 15 THEN CAST(metadata_df._6 as int)+1 ELSE CAST(metadata_df._6 as int) END) 
as Adjusted_Month_1,
CAST(metadata_df._7 as int) as Day,
CAST(metadata_df._8 as int) as Year,
(CASE WHEN (CASE WHEN CAST(metadata_df._7 as int) > 15 THEN CAST(metadata_df._6 as int)+1 
ELSE CAST(metadata_df._6 as int) END) > 12 THEN CAST(metadata_df._8 as int)+1 ELSE CAST(metadata_df._8 as int) END) 
as Adjusted_Year,
(CASE WHEN 
(CASE WHEN CAST(metadata_df._7 as int) > 15 THEN CAST(metadata_df._6 as int)+1 ELSE CAST(metadata_df._6 as int) END) 
> 12
THEN 
(CASE WHEN CAST(metadata_df._7 as int) > 15 THEN CAST(metadata_df._6 as int)+1 ELSE CAST(metadata_df._6 as int) END)-12 
ELSE 
(CASE WHEN CAST(metadata_df._7 as int) > 15 THEN CAST(metadata_df._6 as int)+1 ELSE CAST(metadata_df._6 as int) END) 
END) as Adjusted_Month_2
FROM metadata_df)

""")
metadata_df_2.registerTempTable("metadata_df_2")
metadata_df_2.show()

+--------------------+------------+--------+---------+---------+--------+-----+----+
|               Title|Release_Date|  Budget|  Revenue|   Profit|Run_Time|Month|Year|
+--------------------+------------+--------+---------+---------+--------+-----+----+
|           Toy Story|  10/30/1995|30000000|373554033|343554033|      81|   11|1995|
|             Jumanji|  12/15/1995|65000000|262797249|197797249|     104|   12|1995|
|    Grumpier Old Men|  12/22/1995|       0|        0|        0|     101|    1|1996|
|   Waiting to Exhale|  12/22/1995|16000000| 81452156| 65452156|     127|    1|1996|
|Father of the Bri...|   2/10/1995|       0| 76578911| 76578911|     106|    2|1995|
|                Heat|  12/15/1995|60000000|187436818|127436818|     170|   12|1995|
|             Sabrina|  12/15/1995|58000000|        0|-58000000|     127|   12|1995|
|        Tom and Huck|  12/22/1995|       0|        0|        0|      97|    1|1996|
|        Sudden Death|  12/22/1995|35000000| 64350171| 29350171| 

In [8]:
input_file = sc.textFile("US CPI.csv")
input_file = input_file.flatMap(lambda line: [line.split(",")[:5]+line.split(",")[0].split("/")])
cpi_df = input_file.toDF()
cpi_df.registerTempTable("cpi_df")
cpi_df.show()
cpi_df.count()

+---------+----+---+---+----+
|       _1|  _2| _3| _4|  _5|
+---------+----+---+---+----+
| 1/1/1913| 9.8|  1|  1|1913|
| 1/2/1913| 9.8|  1|  2|1913|
| 1/3/1913| 9.8|  1|  3|1913|
| 1/4/1913| 9.8|  1|  4|1913|
| 1/5/1913| 9.7|  1|  5|1913|
| 1/6/1913| 9.8|  1|  6|1913|
| 1/7/1913| 9.9|  1|  7|1913|
| 1/8/1913| 9.9|  1|  8|1913|
| 1/9/1913|  10|  1|  9|1913|
|1/10/1913|  10|  1| 10|1913|
|1/11/1913|10.1|  1| 11|1913|
|1/12/1913|  10|  1| 12|1913|
| 1/1/1914|  10|  1|  1|1914|
| 1/2/1914| 9.9|  1|  2|1914|
| 1/3/1914| 9.9|  1|  3|1914|
| 1/4/1914| 9.8|  1|  4|1914|
| 1/5/1914| 9.9|  1|  5|1914|
| 1/6/1914| 9.9|  1|  6|1914|
| 1/7/1914|  10|  1|  7|1914|
| 1/8/1914|10.2|  1|  8|1914|
+---------+----+---+---+----+
only showing top 20 rows



1303

In [9]:
cpi_df_1 = sqlc.sql("""
SELECT  
cpi_df._1 as Date, 
CAST(cpi_df._2 as double) as CPI, 
CAST(cpi_df._3 as int) as Day,
CAST(cpi_df._4 as int) as Month,
CAST(cpi_df._5 as int) as Year
FROM cpi_df
""")
cpi_df_1.registerTempTable("cpi_df_1")
cpi_df_1.show()

+---------+----+---+-----+----+
|     Date| CPI|Day|Month|Year|
+---------+----+---+-----+----+
| 1/1/1913| 9.8|  1|    1|1913|
| 1/2/1913| 9.8|  1|    2|1913|
| 1/3/1913| 9.8|  1|    3|1913|
| 1/4/1913| 9.8|  1|    4|1913|
| 1/5/1913| 9.7|  1|    5|1913|
| 1/6/1913| 9.8|  1|    6|1913|
| 1/7/1913| 9.9|  1|    7|1913|
| 1/8/1913| 9.9|  1|    8|1913|
| 1/9/1913|10.0|  1|    9|1913|
|1/10/1913|10.0|  1|   10|1913|
|1/11/1913|10.1|  1|   11|1913|
|1/12/1913|10.0|  1|   12|1913|
| 1/1/1914|10.0|  1|    1|1914|
| 1/2/1914| 9.9|  1|    2|1914|
| 1/3/1914| 9.9|  1|    3|1914|
| 1/4/1914| 9.8|  1|    4|1914|
| 1/5/1914| 9.9|  1|    5|1914|
| 1/6/1914| 9.9|  1|    6|1914|
| 1/7/1914|10.0|  1|    7|1914|
| 1/8/1914|10.2|  1|    8|1914|
+---------+----+---+-----+----+
only showing top 20 rows



In [183]:
metadata_df_4 = sqlc.sql("""
SELECT Title, Release_Date, Adjusted_Budget, Adjusted_Revenue, Adjusted_Profit FROM
(SELECT A.Title as Title, A.Release_Date as Release_Date, A.Budget as Budget,A.Revenue as Revenue,A.Profit as Profit, 
A.Run_Time as Run_Time,
B.CPI as CPI,
A.Budget/B.CPI as Adjusted_Budget,
A.Revenue/B.CPI as Adjusted_Revenue,
A.Profit/B.CPI as Adjusted_Profit
FROM
metadata_df_2 as A
LEFT JOIN cpi_df_1 as B
ON A.Year = B.Year AND A.Month = B.MONTH)
""")
metadata_df_4.registerTempTable("metadata_df_4")
metadata_df_4.show(20)

+--------------------+------------+------------------+------------------+-------------------+
|               Title|Release_Date|   Adjusted_Budget|  Adjusted_Revenue|    Adjusted_Profit|
+--------------------+------------+------------------+------------------+-------------------+
|             Jumanji|  12/15/1995| 423452.7687296417|1712034.1954397394| 1288581.4267100978|
|                Heat|  12/15/1995|390879.47882736154|1221086.7622149838|  830207.2833876221|
|             Sabrina|  12/15/1995| 377850.1628664495|               0.0| -377850.1628664495|
|           GoldenEye|  11/16/1995| 377850.1628664495| 2294423.674267101| 1916573.5114006514|
|The American Pres...|  11/17/1995|403908.79478827363|  702798.019543974|  298889.2247557003|
|              Casino|  11/22/1995|338762.21498371335| 756432.4104234527|  417670.1954397394|
|Sense and Sensibi...|  12/13/1995|107491.85667752443| 879478.8273615636|   771986.970684039|
|          Four Rooms|   12/9/1995|26058.631921824104|28013.

In [184]:
metadata_df_4.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Release_Date: string (nullable = true)
 |-- Adjusted_Budget: double (nullable = true)
 |-- Adjusted_Revenue: double (nullable = true)
 |-- Adjusted_Profit: double (nullable = true)



In [ ]:
# from command prompt
pyspark --packages com.databricks:spark-csv_2.10:1.4.0

# on pyspark 
 schema1 = StructType ([ StructField("id",IntegerType(), True), \
         StructField("cat_id",IntegerType(), True), \
         StructField("name",StringType(), True),\
         StructField("desc",StringType(), True),\
         StructField("price",DecimalType(), True), \
         StructField("url",StringType(), True)
         ])

In [185]:
metadata_df_5 = sqlc.sql("""
SELECT Title, Release_Date, Budget, Revenue, Profit FROM
(SELECT A.Title as Title, A.Release_Date as Release_Date, A.Budget as Budget,A.Revenue as Revenue,A.Profit as Profit, 
A.Run_Time as Run_Time,
B.CPI as CPI,
A.Budget/B.CPI as Adjusted_Budget,
A.Revenue/B.CPI as Adjusted_Revenue,
A.Profit/B.CPI as Adjusted_Profit
FROM
metadata_df_2 as A
LEFT JOIN cpi_df_1 as B
ON A.Year = B.Year AND A.Month = B.MONTH)
""")
metadata_df_5.registerTempTable("metadata_df_5")
metadata_df_5.show(20)

+--------------------+------------+--------+---------+---------+
|               Title|Release_Date|  Budget|  Revenue|   Profit|
+--------------------+------------+--------+---------+---------+
|             Jumanji|  12/15/1995|65000000|262797249|197797249|
|                Heat|  12/15/1995|60000000|187436818|127436818|
|             Sabrina|  12/15/1995|58000000|        0|-58000000|
|           GoldenEye|  11/16/1995|58000000|352194034|294194034|
|The American Pres...|  11/17/1995|62000000|107879496| 45879496|
|              Casino|  11/22/1995|52000000|116112375| 64112375|
|Sense and Sensibi...|  12/13/1995|16500000|135000000|118500000|
|          Four Rooms|   12/9/1995| 4000000|  4300000|   300000|
|         Money Train|  11/21/1995|60000000| 35431113|-24568887|
|           Toy Story|  10/30/1995|30000000|373554033|343554033|
|Ace Ventura: When...|  11/10/1995|30000000|212385533|182385533|
|          Get Shorty|  10/20/1995|30250000|115101622| 84851622|
|Father of the Bri...|   

In [192]:
metadata_df_6 = sqlc.sql("""
SELECT Title, Release_Date, Adjusted_Profit FROM
metadata_df_4
ORDER BY Adjusted_Profit DESC
""")
metadata_df_6.show(200)

+--------------------+------------+--------------------+
|               Title|Release_Date|     Adjusted_Profit|
+--------------------+------------+--------------------+
|  Gone with the Wind|  12/15/1939|        2.82983185E7|
| Alice in Wonderland|    7/3/1951| 2.196911196911197E7|
|               Bambi|   8/14/1942|1.6156918181818182E7|
|Snow White and th...|  12/20/1937|1.2918103028169015E7|
|           Star Wars|   5/25/1977|1.2593047891268533E7|
|          Cinderella|    3/4/1950|1.1046246398305085E7|
|             Titanic|  11/18/1997|1.0198600049597023E7|
|        The Exorcist|  12/26/1973|   9298415.128755365|
|  The Sound of Music|    3/2/1965|   8882245.559105432|
|                Jaws|   6/18/1975|    8554501.84501845|
|E.T. the Extra-Te...|    4/3/1982|   8245156.227608008|
|Star Wars: The Fo...|  12/15/2015|   7708375.960257901|
|One Hundred and O...|   1/25/1961|   7110067.583892617|
|The Empire Strike...|   5/17/1980|  6292623.9419588875|
|     The Jungle Book|  10/17/1

In [193]:
metadata_df_7 = sqlc.sql("""
SELECT Title, Release_Date, Profit FROM
metadata_df_5
ORDER BY Profit DESC
""")
metadata_df_7.show(200)

+--------------------+------------+----------+
|               Title|Release_Date|    Profit|
+--------------------+------------+----------+
|Star Wars: The Fo...|  12/15/2015|1823223624|
|             Titanic|  11/18/1997|1645034188|
|      Jurassic World|    6/9/2015|1363528810|
|           Furious 7|    4/1/2015|1316249360|
|        The Avengers|   4/25/2012|1299557910|
|Harry Potter and ...|    7/7/2011|1217000000|
|Avengers: Age of ...|   4/22/2015|1125403694|
|              Frozen|  11/27/2013|1124219009|
|Beauty and the Beast|   3/16/2017|1102886337|
|             Minions|   6/17/2015|1082730962|
|The Lord of the R...|   12/1/2003|1024888979|
|          Iron Man 3|   4/18/2013|1015439994|
|The Fate of the F...|   4/12/2017| 988764765|
|     Despicable Me 3|   6/15/2017| 940063384|
|Transformers: Dar...|   6/28/2011| 928746996|
|             Skyfall|  10/25/2012| 908561013|
|Captain America: ...|   4/27/2016| 903304495|
|     Despicable Me 2|   6/25/2013| 894761885|
|Transformers

In [194]:
metadata_df_8 = sqlc.sql("""
SELECT Title, Release_Date, Profit FROM
metadata_df_5
ORDER BY Profit ASC
""")
metadata_df_8.show(200)

+--------------------+------------+----------+
|               Title|Release_Date|    Profit|
+--------------------+------------+----------+
|              Avatar|  12/10/2009|      null|
|     The Lone Ranger|    7/3/2013|-165710090|
|         The Wolfman|   2/11/2010|-150000000|
|    Band of Brothers|    9/9/2001|-125000000|
|         The Pacific|   3/15/2010|-120000000|
|           The Alamo|    4/7/2004|-119180039|
|     Mars Needs Moms|    3/9/2011|-111007242|
|Valerian and the ...|   7/20/2017|-107447384|
|      Double Wedding|   7/20/2010|-104002432|
|         Война и мир|   3/14/1966|-100000000|
|        Brother Bear|  10/23/2003|-100000000|
|Dragonball Evolution|    4/1/2009|-100000000|
|    The 13th Warrior|   8/27/1999| -98301101|
|The Adventures of...|   8/15/2002| -92896027|
|          Metropolis|   1/10/1927| -91969578|
|             Volcano|   4/25/1997| -90000000|
|    Mighty Joe Young|  12/25/1998| -90000000|
|Payback: Straight Up|  10/21/2006| -90000000|
|    Cutthroa

In [4]:
input_file = sc.textFile("movies_metadata_2.csv")
input_file = input_file.flatMap(lambda line: [line.split(",")[:8]])
input_file = input_file.filter(lambda line: line[0].isnumeric() and int(line[0])!=0 and line[3].isnumeric() and int(line[3])!=0 and line[4].isnumeric() and line[5].isnumeric() and line[6].isnumeric() and line[7].isnumeric())
metadata_df = input_file.toDF()
metadata_df.registerTempTable("metadata_df")
metadata_df.show(20)
metadata_df.count()

/sw/pkgs/arc/spark/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


+--------+--------------------+----------+---------+---+---+---+----+
|      _1|                  _2|        _3|       _4| _5| _6| _7|  _8|
+--------+--------------------+----------+---------+---+---+---+----+
|30000000|           Toy Story|10/30/1995|373554033| 81| 10| 30|1995|
|65000000|             Jumanji|12/15/1995|262797249|104| 12| 15|1995|
|16000000|   Waiting to Exhale|12/22/1995| 81452156|127| 12| 22|1995|
|60000000|                Heat|12/15/1995|187436818|170| 12| 15|1995|
|35000000|        Sudden Death|12/22/1995| 64350171|106| 12| 22|1995|
|58000000|           GoldenEye|11/16/1995|352194034|130| 11| 16|1995|
|62000000|The American Pres...|11/17/1995|107879496|106| 11| 17|1995|
|44000000|               Nixon|12/22/1995| 13681765|192| 12| 22|1995|
|98000000|    Cutthroat Island|12/22/1995| 10017322|119| 12| 22|1995|
|52000000|              Casino|11/22/1995|116112375|178| 11| 22|1995|
|16500000|Sense and Sensibi...|12/13/1995|135000000|136| 12| 13|1995|
| 4000000|          

5257

In [20]:
metadata_df_2 = sqlc.sql("""
SELECT Title,Release_Date,Budget,Revenue,Profit,Return_Rate,Run_Time,Adjusted_Month_2 as Month,Adjusted_Year as Year FROM
(SELECT  
metadata_df._2 as Title, 
metadata_df._3 as Release_Date, 
CAST(metadata_df._1 as int) as Budget,
CAST(metadata_df._4 as int) as Revenue,
CAST(metadata_df._4 as int)-CAST(metadata_df._1 as int) as Profit,
CAST(metadata_df._4 as int)/CAST(metadata_df._1 as int) as Return_Rate,
CAST(metadata_df._5 as int) as Run_Time,
CAST(_6 as int) as Month,
(CASE WHEN CAST(metadata_df._7 as int) > 15 THEN CAST(metadata_df._6 as int)+1 ELSE CAST(metadata_df._6 as int) END) 
as Adjusted_Month_1,
CAST(metadata_df._7 as int) as Day,
CAST(metadata_df._8 as int) as Year,
(CASE WHEN (CASE WHEN CAST(metadata_df._7 as int) > 15 THEN CAST(metadata_df._6 as int)+1 
ELSE CAST(metadata_df._6 as int) END) > 12 THEN CAST(metadata_df._8 as int)+1 ELSE CAST(metadata_df._8 as int) END) 
as Adjusted_Year,
(CASE WHEN 
(CASE WHEN CAST(metadata_df._7 as int) > 15 THEN CAST(metadata_df._6 as int)+1 ELSE CAST(metadata_df._6 as int) END) 
> 12
THEN 
(CASE WHEN CAST(metadata_df._7 as int) > 15 THEN CAST(metadata_df._6 as int)+1 ELSE CAST(metadata_df._6 as int) END)-12 
ELSE 
(CASE WHEN CAST(metadata_df._7 as int) > 15 THEN CAST(metadata_df._6 as int)+1 ELSE CAST(metadata_df._6 as int) END) 
END) as Adjusted_Month_2
FROM metadata_df)

""")
metadata_df_2.registerTempTable("metadata_df_2")
metadata_df_2.show()

+--------------------+------------+--------+---------+---------+-------------------+--------+-----+----+
|               Title|Release_Date|  Budget|  Revenue|   Profit|        Return_Rate|Run_Time|Month|Year|
+--------------------+------------+--------+---------+---------+-------------------+--------+-----+----+
|           Toy Story|  10/30/1995|30000000|373554033|343554033|         12.4518011|      81|   11|1995|
|             Jumanji|  12/15/1995|65000000|262797249|197797249|          4.0430346|     104|   12|1995|
|   Waiting to Exhale|  12/22/1995|16000000| 81452156| 65452156|         5.09075975|     127|    1|1996|
|                Heat|  12/15/1995|60000000|187436818|127436818|  3.123946966666667|     170|   12|1995|
|        Sudden Death|  12/22/1995|35000000| 64350171| 29350171| 1.8385763142857143|     106|    1|1996|
|           GoldenEye|  11/16/1995|58000000|352194034|294194034|  6.072310931034483|     130|   12|1995|
|The American Pres...|  11/17/1995|62000000|107879496| 

In [22]:
metadata_df_4 = sqlc.sql("""
SELECT Title, Release_Date, Budget, Revenue, Profit,Return_Rate, Adjusted_Budget, Adjusted_Revenue, Adjusted_Profit FROM
(SELECT A.Title as Title, A.Release_Date as Release_Date, A.Budget as Budget,A.Revenue as Revenue,A.Profit as Profit, A.Return_Rate as Return_Rate,
A.Run_Time as Run_Time,
B.CPI as CPI,
A.Budget/B.CPI as Adjusted_Budget,
A.Revenue/B.CPI as Adjusted_Revenue,
A.Profit/B.CPI as Adjusted_Profit
FROM
metadata_df_2 as A
LEFT JOIN cpi_df_1 as B
ON A.Year = B.Year AND A.Month = B.MONTH)
""")
metadata_df_4.registerTempTable("metadata_df_4")
metadata_df_4.show(20)

+--------------------+------------+---------+---------+---------+--------------------+------------------+------------------+-------------------+
|               Title|Release_Date|   Budget|  Revenue|   Profit|         Return_Rate|   Adjusted_Budget|  Adjusted_Revenue|    Adjusted_Profit|
+--------------------+------------+---------+---------+---------+--------------------+------------------+------------------+-------------------+
| Across to Singapore|    4/7/1928|   290000|   596000|   306000|  2.0551724137931036| 16959.06432748538| 34853.80116959064| 17894.736842105263|
|Captain America: ...|   3/20/2014|170000000|714766572|544766572|   4.204509247058824| 717081.7304447594|3014976.7665519337|  2297895.036107174|
|                Noah|   3/20/2014|125000000|362637473|237637473|         2.901099784| 527265.9782682054|1529651.2156644394| 1002385.2373962341|
| Muppets Most Wanted|   3/20/2014| 50000000| 80383290| 30383290|           1.6076658|210906.39130728217|339066.99230613484| 12816

In [24]:
metadata_df_13 = sqlc.sql("""
SELECT Title, Release_Date, Budget, Revenue, Profit, Return_Rate FROM
metadata_df_4
ORDER BY Return_Rate DESC
""")
metadata_df_13.show(100)
metadata_df_13.coalesce(1).write.csv("removal Return_Rate DESC.csv")
metadata_df_13

+---------------------+------------+--------+---------+---------+------------------+
|                Title|Release_Date|  Budget|  Revenue|   Profit|       Return_Rate|
+---------------------+------------+--------+---------+---------+------------------+
|       Less Than Zero|   11/6/1987|       1| 12396383| 12396382|       1.2396383E7|
|         Modern Times|    2/5/1936|       1|  8500000|  8499999|         8500000.0|
|       Aquí Entre Nos|   3/30/2012|       1|  2755584|  2755583|         2755584.0|
|            Nurse 3-D|   9/28/2013|      10| 10000000|  9999990|         1000000.0|
|   From Prada to Nada|   1/28/2011|      93|  2500000|  2499907|26881.720430107525|
|  Paranormal Activity|   9/14/2007|   15000|193355800|193340800|12890.386666666667|
|            Tarnation|  10/19/2003|     218|  1162014|  1161796| 5330.339449541284|
| The Blair Witch P...|   7/14/1999|   60000|248000000|247940000| 4133.333333333333|
|"대�?""While the K...|  12/16/2015|    5000| 11083449| 11078449| 

DataFrame[Title: string, Release_Date: string, Budget: int, Revenue: int, Profit: int, Return_Rate: double]

In [11]:
metadata_df_6 = sqlc.sql("""
SELECT Title, Release_Date, Budget, Revenue, Profit, Adjusted_Profit FROM
metadata_df_4
ORDER BY Adjusted_Profit DESC
""")
metadata_df_6.show(100)
metadata_df_6.coalesce(1).write.csv("removal Adjusted_Profit DESC.csv")
metadata_df_6

+--------------------+------------+---------+----------+----------+--------------------+
|               Title|Release_Date|   Budget|   Revenue|    Profit|     Adjusted_Profit|
+--------------------+------------+---------+----------+----------+--------------------+
|  Gone with the Wind|  12/15/1939|  4000000| 400176459| 396176459|        2.82983185E7|
| Alice in Wonderland|    7/3/1951|  3000000| 572000000| 569000000| 2.196911196911197E7|
|               Bambi|   8/14/1942|   858000| 267447150| 266589150|1.6156918181818182E7|
|Snow White and th...|  12/20/1937|  1488423| 184925486| 183437063|1.2918103028169015E7|
|           Star Wars|   5/25/1977| 11000000| 775398007| 764398007|1.2593047891268533E7|
|          Cinderella|    3/4/1950|  2900000| 263591415| 260691415|1.1046246398305085E7|
|             Titanic|  11/18/1997|200000000|1845034188|1645034188|1.0198600049597023E7|
|        The Exorcist|  12/26/1973|  8000000| 441306145| 433306145|   9298415.128755365|
|  The Sound of Music

AnalysisException: path file:/home/yifeisun/removal Adjusted_Profit DESC.csv already exists.

In [12]:
metadata_df_7 = sqlc.sql("""
SELECT Title, Release_Date, Budget, Revenue, Profit, Adjusted_Profit FROM
metadata_df_4
ORDER BY Profit DESC
""")
metadata_df_7.coalesce(1).write.csv("removal Profit DESC.csv")
metadata_df_7.show(100)

AnalysisException: path file:/home/yifeisun/removal Profit DESC.csv already exists.

In [13]:
metadata_df_8 = sqlc.sql("""
SELECT Title, Release_Date, Budget, Revenue, Profit, Adjusted_Profit FROM
metadata_df_4
ORDER BY Profit ASC
""")
metadata_df_8.coalesce(1).write.csv("removal Profit ASC.csv")
metadata_df_8.show(100)

AnalysisException: path file:/home/yifeisun/removal Profit ASC.csv already exists.

In [14]:
metadata_df_9 = sqlc.sql("""
SELECT Title, Release_Date, Budget, Revenue, Profit, Adjusted_Profit FROM
metadata_df_4
ORDER BY Adjusted_Profit ASC
""")
metadata_df_9.coalesce(1).write.csv("removal Adjusted_Profit ASC.csv")
metadata_df_9.show(100)

AnalysisException: path file:/home/yifeisun/removal Adjusted_Profit ASC.csv already exists.

In [15]:
metadata_df_10 = sqlc.sql("""
SELECT Title, Release_Date, Budget, Revenue, Profit, Adjusted_Budget FROM
metadata_df_4
ORDER BY Budget DESC
""")
metadata_df_10.coalesce(1).write.csv("removal Budget DESC.csv")
metadata_df_10.show(100)

+--------------------+------------+---------+----------+----------+------------------+
|               Title|Release_Date|   Budget|   Revenue|    Profit|   Adjusted_Budget|
+--------------------+------------+---------+----------+----------+------------------+
|Pirates of the Ca...|   5/14/2011|380000000|1045713802| 665713802| 1681683.808040219|
|Pirates of the Ca...|   5/19/2007|300000000| 961000000| 661000000|1439870.9875595146|
|Avengers: Age of ...|   4/22/2015|280000000|1405403694|1125403694|1177435.2936229263|
|    Superman Returns|   6/28/2006|270000000| 391081192| 121081192|1326781.3267813267|
|             Tangled|  11/24/2010|260000000| 591794936| 331794936| 1186245.032598926|
|         John Carter|    3/7/2012|260000000| 284139100|  24139100| 1133430.982771849|
|Transformers: The...|   6/21/2017|260000000| 604942143| 344942143|1062152.2472690432|
|        Spider-Man 3|    5/1/2007|258000000| 890871626| 632871626|1240688.8227401911|
|     The Lone Ranger|    7/3/2013|25500000

In [16]:
metadata_df_10 = sqlc.sql("""
SELECT Title, Release_Date, Budget, Revenue, Profit, Adjusted_Budget FROM
metadata_df_4
ORDER BY Adjusted_Budget DESC
""")
metadata_df_10.coalesce(1).write.csv("removal Adjusted_Budget DESC.csv")
metadata_df_10.show(100)

+--------------------+------------+---------+----------+----------+------------------+
|               Title|Release_Date|   Budget|   Revenue|    Profit|   Adjusted_Budget|
+--------------------+------------+---------+----------+----------+------------------+
|          Metropolis|   1/10/1927| 92620000|    650422| -91969578| 5292571.428571428|
|The Manchurian Ca...|  10/24/1962| 80000000|  96105910|  16105910| 2631578.947368421|
|Pirates of the Ca...|   5/14/2011|380000000|1045713802| 665713802| 1681683.808040219|
|Pirates of the Ca...|   5/19/2007|300000000| 961000000| 661000000|1439870.9875595146|
|    Superman Returns|   6/28/2006|270000000| 391081192| 121081192|1326781.3267813267|
|        Spider-Man 3|    5/1/2007|258000000| 890871626| 632871626|1240688.8227401911|
|             Titanic|  11/18/1997|200000000|1845034188|1645034188| 1239925.604463732|
|             Tangled|  11/24/2010|260000000| 591794936| 331794936| 1186245.032598926|
|Avengers: Age of ...|   4/22/2015|28000000

In [17]:
metadata_df_11 = sqlc.sql("""
SELECT Title, Release_Date, Budget, Revenue, Profit, Adjusted_Revenue FROM
metadata_df_4
ORDER BY Revenue DESC
""")
metadata_df_11.coalesce(1).write.csv("removal Revenue DESC.csv")
metadata_df_11.show(100)

+--------------------+------------+---------+----------+----------+--------------------+
|               Title|Release_Date|   Budget|   Revenue|    Profit|    Adjusted_Revenue|
+--------------------+------------+---------+----------+----------+--------------------+
|Star Wars: The Fo...|  12/15/2015|245000000|2068223624|1823223624|   8744207.267730683|
|             Titanic|  11/18/1997|200000000|1845034188|1645034188|1.1438525654060755E7|
|        The Avengers|   4/25/2012|220000000|1519557910|1299557910|   6612091.943519788|
|      Jurassic World|    6/9/2015|150000000|1513528810|1363528810|   6342362.951415952|
|           Furious 7|    4/1/2015|190000000|1506249360|1316249360|   6366254.126179739|
|Avengers: Age of ...|   4/22/2015|280000000|1405403694|1125403694|   5909899.682512983|
|Harry Potter and ...|    7/7/2011|125000000|1342000000|1217000000|   5940103.221465816|
|              Frozen|  11/27/2013|150000000|1274219009|1124219009|   5467601.272693725|
|Beauty and the Beast

In [18]:
metadata_df_12 = sqlc.sql("""
SELECT Title, Release_Date, Budget, Revenue, Profit, Adjusted_Revenue FROM
metadata_df_4
ORDER BY Adjusted_Revenue DESC
""")
metadata_df_12.coalesce(1).write.csv("removal Adjusted_Revenue DESC.csv")
metadata_df_12.show(100)

+--------------------+------------+---------+----------+----------+--------------------+
|               Title|Release_Date|   Budget|   Revenue|    Profit|    Adjusted_Revenue|
+--------------------+------------+---------+----------+----------+--------------------+
|  Gone with the Wind|  12/15/1939|  4000000| 400176459| 396176459|2.8584032785714287E7|
| Alice in Wonderland|    7/3/1951|  3000000| 572000000| 569000000|2.2084942084942088E7|
|               Bambi|   8/14/1942|   858000| 267447150| 266589150|1.6208918181818182E7|
|Snow White and th...|  12/20/1937|  1488423| 184925486| 183437063|1.3022921549295776E7|
|           Star Wars|   5/25/1977| 11000000| 775398007| 764398007|1.2774267001647446E7|
|             Titanic|  11/18/1997|200000000|1845034188|1645034188|1.1438525654060755E7|
|          Cinderella|    3/4/1950|  2900000| 263591415| 260691415|1.1169127754237287E7|
|        The Exorcist|  12/26/1973|  8000000| 441306145| 433306145|   9470088.948497854|
|  The Sound of Music